In [ ]:
from model.make_model import *

wandb.init(project="dtu-course", entity="freja-thoresen")

figure_path = '../figures/'

use_cuda=False

cuda=False
if torch.cuda.is_available():
  dev = "cuda:0"
  cuda=True
else:
  dev = "cpu"
device = torch.device(dev)

sequence_length = 10
epochs = 5 # 1000 epochs
learning_rate = 0.002
lstm_dropbout = 0.2
linear_dropout = 0.2
input_size = 1 # number of features
lstm_hidden_size = 20 # number of features in hidden state
linear_hidden_size = 40 # number of features in hidden state
num_layers = 2 # number of stacked lstm layers
num_classes = 1 # number of output classes
no_batches = 100
no_files = 100 # no. data files

wandb.config = {
    "learning_rate": learning_rate,
    "sequence_length": sequence_length,
    "epochs": epochs,
    "input_size": input_size,
    "lstm_hidden_size": lstm_hidden_size,
    "linear_hidden_size": linear_hidden_size,
    "num_layers": num_layers,
    "no_batches": no_batches,
    "no_files": no_files,
    "lstm_dropbout": lstm_dropbout,
    "linear_dropout": linear_dropout,
    "num_classes": num_classes,
    "input_size": input_size
}

sweep_config = {
    'name': 'model-sweep',
    'method': 'grid',
    'metric': {
        'name': 'training_loss',
        'goal': 'minimize'
    }
}
parameters_dict = {
    'learning_rate': {
        'value': 0.0002
    },
    'sequence_length': {
        'value': 10
    },
    'epochs': {
        'value': 40
    },
    'input_size': {
        'value': input_size
    },
    'lstm_hidden_size': {
        'values': [20, 40, 60]
    },
    'linear_hidden_size': {
        'values': [20, 40, 60]
    },
    'num_layers': {
        'values': [2, 3]
    },
    'no_batches': {
        'value': 100
    },
    'no_files': {
        'value': no_files
    },
    'lstm_dropbout': {
        'values': [0.2, 0.4]
    },
    'linear_dropout': {
        'values': [0.2, 0.4]
    },
    'num_classes': {
        'value': num_classes
    }
}

sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project="dtu-course")

def sweep():
    with wandb.init() as run:
        config = wandb.config
        lstm, criterion, optimizer = make_model(config)
        train_model(lstm, criterion, optimizer, config)
wandb.agent(sweep_id, function=sweep)